In [3]:
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger

# import os
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = "max_split_size_mb:128"
import torch
import numpy as np
import pandas as pd
import pickle

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

c:\Users\sonng\anaconda3\envs\tsff_env\lib\site-packages\pytorch_forecasting\models\base_model.py:30: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
import warnings
# To ignore all FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", ".*does not have many workers.*")
warnings.filterwarnings("ignore", ".*and is already saved during checkpointing*")
warnings.filterwarnings("ignore", ".*The number of training batches*")

import logging
logging.getLogger("lightning").setLevel(logging.ERROR)
logging.getLogger("pytorch_lightning.utilities.rank_zero").setLevel(logging.ERROR)
logging.getLogger("pytorch_lightning.accelerators.cuda").setLevel(logging.ERROR)


In [1]:
from ESRNN.m4_data import *
from ESRNN.utils_evaluation import evaluate_prediction_owa
from ESRNN.utils_visualization import plot_grid_prediction

In [2]:
X_train_df, y_train_df, X_test_df, y_test_df = prepare_m4_data(dataset_name="Hourly",
                                                               directory="../../data/M4",
                                                               num_obs=414)

In [6]:
unique_ids = y_train_df['unique_id'].unique()

In [7]:
all_forecasts = {}

In [8]:
pl.seed_everything(42)

Seed set to 42


42

In [9]:
df_arima_train = pd.read_csv('../../results/m4/base_model_train_set/y_hat_df_arima_ts.csv')
df_theta_train = pd.read_csv('../../results/m4/base_model_train_set/y_hat_df_theta_ts.csv')
df_xgb_train = pd.read_csv('../../results/m4/base_model_train_set/y_hat_df_xgb_ts.csv')
df_gru_train = pd.read_csv('../../results/m4/base_model_train_set/y_hat_df_gru_ts.csv')
df_lstm_train = pd.read_csv('../../results/m4/base_model_train_set/y_hat_df_lstm_ts.csv')

In [10]:
df_arima_test = pd.read_csv('../../results/m4/y_hat_df_arima.csv')
df_theta_test = pd.read_csv('../../results/m4/y_hat_df_theta.csv')
df_xgb_test = pd.read_csv('../../results/m4/y_hat_df_xgb.csv')
df_gru_test = pd.read_csv('../../results/m4/y_hat_df_gru.csv')
df_lstm_test = pd.read_csv('../../results/m4/y_hat_df_lstm.csv')

In [10]:
for unique_id in ['H1']:

    print(f'Currently training: {unique_id}')

    df_base_models_train= pd.DataFrame({
    'unique_id' : df_arima_train.unique_id,
    'y_arima' : df_arima_train.y_hat,
    'y_theta' : df_theta_train.y_hat,
    'y_xgb' : df_xgb_train.y,
    'y_gru' : df_gru_train.y_hat,
    'y_lstm' : df_lstm_train.y_hat
    })

    df_base_models_test= pd.DataFrame({
    'unique_id' : df_arima_test.unique_id,
    'y_arima' : df_arima_test.y_hat,
    'y_theta' : df_theta_test.y_hat,
    'y_xgb' : df_xgb_test.y_hat,
    'y_gru' : df_gru_test.y_hat,
    'y_lstm' : df_lstm_test.y_hat
    })

    # Filter data for the current series (train and val data)
    df = y_train_df[y_train_df['unique_id'] == unique_id].copy()
    df['ds'] = (df['ds'] - df['ds'].min()).dt.total_seconds() // 3600
    df['ds'] = df['ds'].astype(int)
    df_train_val = pd.concat([df.iloc[-24*7:].reset_index(drop=True).drop(columns=['unique_id']), 
                              df_base_models_train[df_base_models_train['unique_id']==unique_id].reset_index(drop=True)], axis=1)
    
    # Test data
    df = y_test_df.drop(columns=['y_hat_naive2'])[y_test_df['unique_id'] == unique_id].copy()
    df['ds'] = (df['ds'] - df['ds'].min()).dt.total_seconds() // 3600 + df_train_val['ds'].max() + 1  #700
    df['ds'] = df['ds'].astype(int)
    df_test = pd.concat([df.reset_index(drop=True).drop(columns=['unique_id']), 
                         df_base_models_test[df_base_models_test['unique_id']==unique_id].reset_index(drop=True)], axis=1)

    # Create the TimeSeriesDataSet for training
    max_encoder_length = 24*7
    max_prediction_length = 48

    training = TimeSeriesDataSet(
        df_train_val.iloc[:-max_prediction_length],
        time_idx="ds",
        target="y",
        group_ids=['unique_id'],
        max_encoder_length=max_encoder_length,
        # min_encoder_length=max_encoder_length // 2,
        min_encoder_length=1,
        max_prediction_length=max_prediction_length,
        # min_prediction_length=max_prediction_length // 2,
        min_prediction_length=1,
        time_varying_known_reals=['y_arima', 'y_theta', 'y_xgb', 'y_gru', 'y_lstm'],  # Base model forecasts
        target_normalizer=GroupNormalizer(
            groups=["unique_id"], transformation="softplus"
        ),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        # allow_missing_timesteps=True
        )
    
    validation = TimeSeriesDataSet.from_dataset(training, df_train_val, predict=True, stop_randomization=True)

    # creating the test data that includes the encoder and decoder data
    encoder_data = df_train_val[lambda x: x.ds > x.ds.max() - max_encoder_length]
    df_test.y = df_train_val.y[df_train_val.ds == df_train_val.ds.max()].values[0]
    decoder_data = df_test
    new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)

    batch_size = 64  # set this between 32 to 128
    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

    ### TUNING
    best_params = {}
    print(f'Tuning has started for {unique_id}...')
    study = optimize_hyperparameters(
        train_dataloader,
        val_dataloader,
        model_path="optuna_tune_test",
        n_trials = 100,
        max_epochs = 50,
        gradient_clip_val_range=(0.1, 1.0),
        hidden_size_range=(50, 200),
        hidden_continuous_size_range=(50, 200),
        attention_head_size_range=(1, 4),
        learning_rate_range=(0.001, 0.1),
        dropout_range=(0.1, 0.3),
        trainer_kwargs=dict(limit_train_batches=30,
                            enable_checkpointing=False,
                            callbacks=[]),
        reduce_on_plateau_patience=4,
        use_learning_rate_finder=False,
    )

    with open(f"best_params/study_{unique_id}.pkl", "wb") as fout:
        pickle.dump(study, fout)

    print(f'Tuning done! Best params for {unique_id}: ')
    print(study.best_trial.params)

    ### END TUNING

    best_params = study.best_trial.params

    # configure network and trainer
    early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-5, patience=50, verbose=False, mode="min")
    lr_logger = LearningRateMonitor()  # log the learning rate
    logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

    trainer = pl.Trainer(
        max_epochs=150,
        accelerator="gpu",
        gradient_clip_val=best_params['gradient_clip_val'],
        limit_train_batches=50,  # coment in for training, running valiation every 30 batches
        # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
        callbacks=[early_stop_callback],
        logger=False,
        enable_model_summary=False,
        enable_checkpointing=False
    )

    tft = TemporalFusionTransformer.from_dataset(
        training,
        learning_rate=best_params['learning_rate'],
        hidden_size=best_params['hidden_size'],
        attention_head_size=best_params['attention_head_size'],
        dropout=best_params['dropout'],
        hidden_continuous_size=best_params['hidden_continuous_size'],
        loss=SMAPE(),
        # log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
        optimizer="Ranger",
        reduce_on_plateau_patience=4,
    )


    trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    )

    new_raw_predictions = tft.predict(new_prediction_data, mode="raw", return_x=True, trainer_kwargs=dict(accelerator="gpu"))
    all_forecasts[unique_id] = new_raw_predictions.output.prediction.cpu().numpy().flatten()



Currently training: H1


[I 2023-11-23 18:41:44,623] A new study created in memory with name: no-name-aff0d309-f724-43dc-816e-451a4a9e5a56


Tuning has started for H1...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[I 2023-11-23 18:42:24,218] Trial 0 finished with value: 37.251129150390625 and parameters: {'gradient_clip_val': 0.44809527270126237, 'hidden_size': 59, 'dropout': 0.13460685794498822, 'hidden_continuous_size': 55, 'attention_head_size': 3, 'learning_rate': 0.004160708133623006}. Best is trial 0 with value: 37.251129150390625.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[I 2023-11-23 18:43:06,023] Trial 1 finished with value: 51.3515625 and parameters: {'gradient_clip_val': 0.5781803227958286, 'hidden_size': 114, 'dropout': 0.19484512474605237, 'hidden_continuous_size': 77, 'attention_head_size': 4, 'learning_rate': 0.08574560213041214}. Best is trial 0 with value: 37.251129150390625.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[I 2023-11-23 18:43:45,005] Trial 2 finished with value: 42.78737258911133 and parameters: {'gradient_clip_val': 0.4552630903730169, 'hidden_size': 89, 'dropout': 0.28160473452343815, 'hidden_continuous_size': 51, 'attentio

Tuning done! Best params for H1: 
{'gradient_clip_val': 0.269168214151623, 'hidden_size': 68, 'dropout': 0.1286910673488292, 'hidden_continuous_size': 52, 'attention_head_size': 3, 'learning_rate': 0.0010083156281266929}
Epoch 94: 100%|██████████| 3/3 [00:00<00:00,  3.97it/s, train_loss_step=0.0359, val_loss=0.0825, train_loss_epoch=0.0356]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [11]:
best_params

{'gradient_clip_val': 0.269168214151623,
 'hidden_size': 68,
 'dropout': 0.1286910673488292,
 'hidden_continuous_size': 52,
 'attention_head_size': 3,
 'learning_rate': 0.0010083156281266929}

In [14]:
best_params

{'gradient_clip_val': 0.01252719132864506,
 'hidden_size': 90,
 'dropout': 0.24957288175755601,
 'hidden_continuous_size': 17,
 'attention_head_size': 3,
 'learning_rate': 0.002420687005025646}

In [12]:
y_hat_new = all_forecasts['H1']

In [23]:
tft_fix = pd.read_csv('../../results/m4/TFT/training/y_hat_df_tft_bm7_fix.csv')

In [24]:
y_hat_fix = tft_fix[tft_fix['unique_id'] == 'H1'].y_hat.to_numpy()

In [25]:
y = y_test_df[y_test_df['unique_id'] == 'H1'].y.to_numpy()

In [24]:
np.sqrt(((y - y_hat_fix)**2).sum())

297.0546158329076

In [25]:
np.sqrt(((y - y_hat_new)**2).sum())

239.7857669789725

In [17]:
y

array([619., 565., 532., 495., 481., 467., 473., 488., 501., 534., 576.,
       639., 712., 772., 830., 880., 893., 896., 891., 854., 803., 769.,
       751., 701., 635., 572., 532., 493., 477., 468., 464., 477., 492.,
       519., 568., 624., 696., 761., 812., 836., 838., 829., 807., 785.,
       756., 719., 703., 659.])

In [18]:
y_hat_fix

array([599.87714, 535.1106 , 492.69775, 458.47858, 433.04303, 418.69598,
       413.63538, 425.68256, 433.354  , 474.0841 , 535.26105, 608.76917,
       680.6441 , 741.88257, 783.6853 , 810.09534, 823.4004 , 828.59406,
       831.62866, 826.578  , 808.09357, 771.691  , 741.979  , 673.0538 ,
       596.1395 , 522.65454, 479.08722, 446.90875, 425.7067 , 414.18658,
       411.84396, 426.32535, 435.20245, 477.3517 , 539.3244 , 612.76495,
       687.0244 , 752.6356 , 797.5754 , 823.9369 , 836.6084 , 839.63513,
       839.9314 , 832.3441 , 813.07574, 775.1058 , 744.4075 , 672.5274 ])

In [19]:
y_hat_new

array([593.6775 , 526.3957 , 490.18463, 455.47583, 431.90118, 423.44656,
       435.80698, 446.98163, 462.7348 , 497.44717, 545.48486, 627.6648 ,
       698.3399 , 758.13745, 797.61975, 825.3072 , 837.12775, 840.80286,
       840.0369 , 831.2204 , 807.3965 , 764.04175, 730.743  , 672.9338 ,
       601.40375, 527.83795, 494.13165, 469.8308 , 434.35284, 419.6026 ,
       428.38242, 449.09546, 463.0781 , 500.19058, 552.2465 , 634.4358 ,
       707.3888 , 761.6421 , 806.7964 , 831.5182 , 845.06537, 849.30615,
       847.30664, 837.69336, 813.26154, 772.75085, 738.3668 , 683.72546],
      dtype=float32)

In [22]:
y - y_hat_fix

array([ 19.12286,  29.8894 ,  39.30225,  36.52142,  47.95697,  48.30402,
        59.36462,  62.31744,  67.646  ,  59.9159 ,  40.73895,  30.23083,
        31.3559 ,  30.11743,  46.3147 ,  69.90466,  69.5996 ,  67.40594,
        59.37134,  27.422  ,  -5.09357,  -2.691  ,   9.021  ,  27.9462 ,
        38.8605 ,  49.34546,  52.91278,  46.09125,  51.2933 ,  53.81342,
        52.15604,  50.67465,  56.79755,  41.6483 ,  28.6756 ,  11.23505,
         8.9756 ,   8.3644 ,  14.4246 ,  12.0631 ,   1.3916 , -10.63513,
       -32.9314 , -47.3441 , -57.07574, -56.1058 , -41.4075 , -13.5274 ])

In [23]:
y - y_hat_new

array([ 25.32250977,  38.60430908,  41.81536865,  39.52416992,
        49.09881592,  43.55343628,  37.19302368,  41.01837158,
        38.26519775,  36.55282593,  30.51513672,  11.33520508,
        13.66009521,  13.86254883,  32.38024902,  54.69281006,
        55.87225342,  55.19714355,  50.96307373,  22.77960205,
        -4.39648438,   4.95825195,  20.25701904,  28.06622314,
        33.59625244,  44.16204834,  37.86834717,  23.16918945,
        42.64715576,  48.3973999 ,  35.61758423,  27.90454102,
        28.92190552,  18.80941772,  15.753479  , -10.43579102,
       -11.38879395,  -0.64208984,   5.20361328,   4.48181152,
        -7.06536865, -20.30615234, -40.30664062, -52.69335938,
       -57.26153564, -53.75085449, -35.36682129, -24.72546387])

In [26]:
y_hat_new_tuned = np.array(
       [580.929  , 500.33313, 453.0575 , 422.3617 , 404.78137, 397.22607,
       395.95535, 402.99728, 410.1695 , 430.85004, 476.80298, 546.1606 ,
       629.58405, 721.15564, 786.5959 , 822.59186, 836.2089 , 837.9536 ,
       835.432  , 828.7916 , 814.7545 , 786.9878 , 748.9407 , 706.76965,
       611.71436, 515.8975 , 464.03384, 430.43143, 410.90222, 402.03238,
       400.5323 , 407.26727, 414.1297 , 435.1363 , 481.22034, 551.06036,
       639.42596, 733.9288 , 797.51306, 829.7677 , 841.40234, 842.2708 ,
       839.2884 , 831.82733, 817.5591 , 789.24927, 751.616  , 709.39325])

In [27]:
np.sqrt(((y - y_hat_fix)**2).sum())

297.0546158329076

In [28]:
np.sqrt(((y - y_hat_new_tuned)**2).sum())

425.1093153632626

In [19]:
tft_fix = pd.read_csv('../../results/m4/TFT/training/y_hat_df_tft_bm7_fix.csv')
y_hat_fix = tft_fix[tft_fix['unique_id'] == 'H2'].y_hat.to_numpy()
y = y_test_df[y_test_df['unique_id'] == 'H2'].y.to_numpy()

In [20]:
y_hat_new_tuned_h2 = np.array(
       [3648.6055, 3198.5742, 2769.7896, 2532.5613, 2367.6936, 2272.463 ,
       2241.8467, 2279.6484, 2402.8403, 2632.213 , 2941.5076, 3264.5334,
       3572.5544, 3819.8118, 3947.2212, 3992.1929, 4000.2188, 3999.1426,
       3996.725 , 3994.8708, 3985.8467, 3961.5388, 3900.3215, 3832.404 ,
       3683.63  , 3252.626 , 2809.6714, 2557.1655, 2405.5974, 2331.3054,
       2321.322 , 2377.0835, 2512.728 , 2751.2598, 3051.8052, 3373.4436,
       3671.6187, 3877.7148, 3973.837 , 4007.355 , 4013.4058, 4015.5168,
       4010.6128, 4008.2886, 4000.525 , 3977.0308, 3913.165 , 3833.0796])

In [21]:
np.sqrt(((y - y_hat_fix)**2).sum())

4816.917033854871

In [22]:
np.sqrt(((y - y_hat_new_tuned_h2)**2).sum())

5390.844602370027

In [30]:
test_dict = {'H1': np.array([580.929  , 500.33313, 453.0575 , 422.3617 , 404.78137, 397.22607,
       395.95535, 402.99728, 410.1695 , 430.85004, 476.80298, 546.1606 ,
       629.58405, 721.15564, 786.5959 , 822.59186, 836.2089 , 837.9536 ,
       835.432  , 828.7916 , 814.7545 , 786.9878 , 748.9407 , 706.76965,
       611.71436, 515.8975 , 464.03384, 430.43143, 410.90222, 402.03238,
       400.5323 , 407.26727, 414.1297 , 435.1363 , 481.22034, 551.06036,
       639.42596, 733.9288 , 797.51306, 829.7677 , 841.40234, 842.2708 ,
       839.2884 , 831.82733, 817.5591 , 789.24927, 751.616  , 709.39325],
      dtype=np.float32), 'H2': np.array([3648.6055, 3198.5742, 2769.7896, 2532.5613, 2367.6936, 2272.463 ,
       2241.8467, 2279.6484, 2402.8403, 2632.213 , 2941.5076, 3264.5334,
       3572.5544, 3819.8118, 3947.2212, 3992.1929, 4000.2188, 3999.1426,
       3996.725 , 3994.8708, 3985.8467, 3961.5388, 3900.3215, 3832.404 ,
       3683.63  , 3252.626 , 2809.6714, 2557.1655, 2405.5974, 2331.3054,
       2321.322 , 2377.0835, 2512.728 , 2751.2598, 3051.8052, 3373.4436,
       3671.6187, 3877.7148, 3973.837 , 4007.355 , 4013.4058, 4015.5168,
       4010.6128, 4008.2886, 4000.525 , 3977.0308, 3913.165 , 3833.0796],
      dtype=np.float32)}

In [51]:
tft_tuned = pd.read_csv('../../results/m4/TFT/test/y_hat_df_tft_bm7_tuned_new.csv')

In [52]:
tft_tuned

,unique_id,ds,y_hat
0,H1,1970-01-30 04:00:00,580.92900
1,H1,1970-01-30 05:00:00,500.33313
2,H1,1970-01-30 06:00:00,453.05750
3,H1,1970-01-30 07:00:00,422.36170
4,H1,1970-01-30 08:00:00,404.78137
...,...,...,...
19867,H99,1970-01-31 23:00:00,24270.91800
19868,H99,1970-02-01 00:00:00,23822.91000
19869,H99,1970-02-01 01:00:00,23406.44000
19870,H99,1970-02-01 02:00:00,23360.03300


In [53]:
evaluate_prediction_owa(tft_tuned, y_train_df, X_test_df, y_test_df, naive2_seasonality=24)

===============  Model evaluation  ==============
OWA: 0.861 
SMAPE: 16.69 
MASE: 1.952 


(0.8614346926341729, 1.951847742892237, 16.69009549604036)

In [3]:
tft_tuned = pd.read_csv('../../results/m4/TFT/test/y_hat_df_tft_bm14_tuned.csv')

In [6]:
y_hat_df = tft_tuned.copy()
y_hat_df['ds'] = X_test_df['ds']
y_hat_df

,unique_id,y_hat,ds
0,H1,619.75390,1970-01-30 04:00:00
1,H1,522.50684,1970-01-30 05:00:00
2,H1,464.61578,1970-01-30 06:00:00
3,H1,426.94696,1970-01-30 07:00:00
4,H1,405.61185,1970-01-30 08:00:00
...,...,...,...
19867,H99,22974.89000,1970-01-31 23:00:00
19868,H99,22505.16800,1970-02-01 00:00:00
19869,H99,22216.00400,1970-02-01 01:00:00
19870,H99,21997.82600,1970-02-01 02:00:00


In [8]:
evaluate_prediction_owa(y_hat_df, y_train_df, X_test_df, y_test_df, naive2_seasonality=24)

===============  Model evaluation  ==============
OWA: 0.795 
SMAPE: 17.43 
MASE: 1.536 


(0.7948093527266985, 1.5362972607125935, 17.430078587235677)